# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [5]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [43]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')
boros.head()

BoroCode       BoroName     Shape_Leng    Shape_Area  \
0         5  Staten Island  330466.075042  1.623827e+09   
1         3       Brooklyn  741185.900596  1.937597e+09   
2         4         Queens  897040.298576  3.045168e+09   
3         1      Manhattan  358408.460709  6.364467e+08   
4         2          Bronx  464400.198868  1.186973e+09   

                                            geometry  
0  (POLYGON ((970217.0223999023 145643.3322143555...  
1  (POLYGON ((1021176.479003906 151374.7969970703...  
2  (POLYGON ((1029606.076599121 156073.8142089844...  
3  (POLYGON ((981219.0557861328 188655.3157958984...  
4  (POLYGON ((1012821.805786133 229228.2645874023...

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [7]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [8]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

altitude  availableBikes  availableDocks city   id landMark  \
0                       22              39       281            
1                       11              22       304            
2                       26              36       359            
3                       20              24       377            
4                       15              19       402            

    lastCommunicationTime   latitude location  longitude postalCode  \
0  2019-03-07 01:06:57 PM  40.764397          -73.973715              
1  2019-03-07 01:03:50 PM  40.704633          -74.013617              
2  2019-03-07 01:06:00 PM  40.755103          -73.974987              
3  2019-03-07 01:05:22 PM  40.722438          -74.005664              
4  2019-03-07 01:05:08 PM  40.740343          -73.989551              

                          stAddress1 stAddress2  \
0  Grand Army Plaza & Central Park S              
1              Broadway & Battery Pl              
2                 E 47 St & Park Ave              
3                   6 Ave & Canal St              
4                 Broadway & E 22 St              

                         stationName  statusKey statusValue  testStation  \
0  Grand Army Plaza & Central Park S          1  In Service        False   
1              Broadway & Battery Pl          1  In Service        False   
2                 E 47 St & Park Ave          1  In Service        False   
3                   6 Ave & Canal St          1  In Service        False   
4                 Broadway & E 22 St          1  In Service        False   

   totalDocks  
0          66  
1          33  
2          64  
3          45  
4          39

In [9]:
# there is one row for each bike station.  How many stations are there? 
len(df)

817

In [10]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

array([-74.0836394 ,  40.65539977, -73.9077436 ,  40.81439444])

In [11]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

array([ 961064.73659426,  178056.37414829, 1009803.53178432,
        235986.42166716])

In [12]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

418 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [13]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp


In [14]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)
len(df)

817

In [17]:
boros

BoroCode       BoroName     Shape_Leng    Shape_Area  \
0         5  Staten Island  330466.075042  1.623827e+09   
1         3       Brooklyn  741185.900596  1.937597e+09   
2         4         Queens  897040.298576  3.045168e+09   
3         1      Manhattan  358408.460709  6.364467e+08   
4         2          Bronx  464400.198868  1.186973e+09   

                                            geometry  
0  (POLYGON ((970217.0223999023 145643.3322143555...  
1  (POLYGON ((1021176.479003906 151374.7969970703...  
2  (POLYGON ((1029606.076599121 156073.8142089844...  
3  (POLYGON ((981219.0557861328 188655.3157958984...  
4  (POLYGON ((1012821.805786133 229228.2645874023...

In [21]:
type(boros)

geopandas.geodataframe.GeoDataFrame

In [18]:
endpoint_url

'http://citibikenyc.com/stations/json'

In [22]:
type(endpoint_url)

str

In [19]:
response

<Response [200]>

In [23]:
type(response)

requests.models.Response

In [20]:
data

{'executionTime': '2019-03-07 01:08:08 PM',
 'stationBeanList': [{'id': 281,
   'stationName': 'Grand Army Plaza & Central Park S',
   'availableDocks': 39,
   'totalDocks': 66,
   'latitude': 40.7643971,
   'longitude': -73.97371465,
   'statusValue': 'In Service',
   'statusKey': 1,
   'availableBikes': 22,
   'stAddress1': 'Grand Army Plaza & Central Park S',
   'stAddress2': '',
   'city': '',
   'postalCode': '',
   'location': '',
   'altitude': '',
   'testStation': False,
   'lastCommunicationTime': '2019-03-07 01:06:57 PM',
   'landMark': ''},
  {'id': 304,
   'stationName': 'Broadway & Battery Pl',
   'availableDocks': 22,
   'totalDocks': 33,
   'latitude': 40.70463334,
   'longitude': -74.01361706,
   'statusValue': 'In Service',
   'statusKey': 1,
   'availableBikes': 11,
   'stAddress1': 'Broadway & Battery Pl',
   'stAddress2': '',
   'city': '',
   'postalCode': '',
   'location': '',
   'altitude': '',
   'testStation': False,
   'lastCommunicationTime': '2019-03-07 01

In [24]:
type(data)

dict

In [44]:
boros.groupby(boros['BoroName']).sum()

BoroCode     Shape_Leng    Shape_Area
BoroName                                            
Bronx                 2  464400.198868  1.186973e+09
Brooklyn              3  741185.900596  1.937597e+09
Manhattan             1  358408.460709  6.364467e+08
Queens                4  897040.298576  3.045168e+09
Staten Island         5  330466.075042  1.623827e+09

In [16]:
manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'availableBikes')

418 availableBikes


In [46]:
print (sum(df['availableBikes'][in_mn]), 'available bikes in Manhattan')

6759 available bikes in Manhattan


In [45]:
brooklyn = boros.geometry[2]
in_bk = df.geometry.within(brooklyn)
print (sum(df['availableBikes'][in_bk]), 'available bikes in Brooklyn')

428 available bikes in Brooklyn


In [29]:
s=boros.BoroName.to_frame()
for index, row in boros.iterrows():
    s[row.BoroName]=boros.geometry.touches(row.geometry)
s

BoroName  Staten Island  Brooklyn  Queens  Manhattan  Bronx
0  Staten Island          False     False   False      False  False
1       Brooklyn          False     False    True       True  False
2         Queens          False      True   False       True   True
3      Manhattan          False      True    True      False   True
4          Bronx          False     False    True       True  False